Data Cleaning & Preparation

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [ ]:
import io

df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

In [ ]:
del df['Symptom_7']
del df['Symptom_8']
del df['Symptom_9']
del df['Symptom_10']
del df['Symptom_11']
del df['Symptom_12']
del df['Symptom_13']
del df['Symptom_14']
del df['Symptom_15']
del df['Symptom_16']
del df['Symptom_17']

In [ ]:
df = df.fillna(' ')

In [ ]:
target_columns = 'Disease'
feature_columns = ['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4', 'Symptom_5', 'Symptom_6']

In [ ]:
def str_lower(s):
  return s.lower()

df[target_columns] = df[target_columns].apply(str_lower)

In [ ]:
def remove_punct(s):
  return s.replace('_', ' ')

for col in feature_columns:
  df[col] = df[col].apply(remove_punct)

In [ ]:
df.to_json('symptoms.json', orient='index')

In [ ]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/dataset.json', 'r+') as f:
  symptoms = json.load(f)

In [ ]:
symptoms

{'intents': [{'tag': 'fungal infection',
   'patterns': ['The patient has the symptoms itching,  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ',
    "The patient's symptoms are itching,  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ",
    'The symptoms are itching,  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ',
    'The patient complains about itching,  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  '],
   'responses': ["It's probably fungal infection",
    'I woudl say the patient is sick of fungal infection',
    'I think you are looking for fungal infection']},
  {'tag': 'fungal infection',
   'patterns': ['The patient has the symptoms  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ,  ',
    "The patient's symptoms are  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ,  ",
    'The symptoms are  skin rash,  nodal skin eruptions,  dischromic  patches,  ,  ,  ',
    'The patient complains about  skin

In [ ]:
sympt = []
for i in symptoms:
  sympt.append([j for j in symptoms[str(i)].values()])

In [ ]:
all_data = []

for k in range(0, len(sympt)):
  sickness = sympt[k][0]
  all_symptoms = ', '.join(sympt[k][1:])
  matched = dict()
  matched['tag'] = sickness
  matched['patterns'] = ['The patient has the symptoms ' + all_symptoms, "The patient's symptoms are " + all_symptoms, "The symptoms are " + all_symptoms, "The patient complains about " + all_symptoms]
  matched['responses'] = ["It's probably " + sickness, "I would say the patient is sick of " + sickness, "I think you are looking for " + sickness]
  all_data.append(matched)

data = dict()
data['intents'] = all_data

AI

In [1]:
import nltk

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.stem import WordNetLemmatizer
import numpy as np
import tensorflow as tf
import json
import pickle
import random

In [3]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/dataset.json', 'r+') as f:
  data = json.load(f)

In [4]:
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in data['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w) for w in words if w not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [6]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
  bag = []
  word_patterns = doc[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

X = list(training[:, 0])
y = list(training[:, 1])

<ipython-input-6-f1630eed4d55>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [ ]:
len(y)

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping

In [59]:
model = Sequential()

model.add(Dense(128, input_shape = (len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128/4, activation='relu'))
"""
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
"""
model.add(Dense(len(y[0]), activation='softmax'))

In [60]:
sgd = SGD(learning_rate=0.01, momentum=0.9)
adamax = Adamax()
early_stopping = EarlyStopping(monitor='accuracy', min_delta=1e-6, patience=15)

In [61]:
model.compile(loss='categorical_crossentropy', optimizer=adamax, metrics=['accuracy'])

In [62]:
hist = model.fit(np.array(X), np.array(y), epochs=200*4, batch_size=8*8, verbose=1, callbacks=early_stopping)

Epoch 1/800
308/308 [==============================] - 2s 2ms/step - loss: 3.6984 - accuracy: 0.0328
Epoch 2/800
308/308 [==============================] - 1s 2ms/step - loss: 3.5109 - accuracy: 0.0577
Epoch 3/800
308/308 [==============================] - 1s 2ms/step - loss: 3.1809 - accuracy: 0.1044
Epoch 4/800
308/308 [==============================] - 1s 3ms/step - loss: 2.9144 - accuracy: 0.1513
Epoch 5/800
308/308 [==============================] - 1s 3ms/step - loss: 2.6755 - accuracy: 0.2007
Epoch 6/800
308/308 [==============================] - 1s 3ms/step - loss: 2.4678 - accuracy: 0.2488
Epoch 7/800
308/308 [==============================] - 1s 2ms/step - loss: 2.2887 - accuracy: 0.2880
Epoch 8/800
308/308 [==============================] - 1s 2ms/step - loss: 2.1234 - accuracy: 0.3183
Epoch 9/800
308/308 [==============================] - 1s 3ms/step - loss: 2.0012 - accuracy: 0.3567
Epoch 10/800
308/308 [==============================] - 1s 3ms/step - loss: 1.8982 - accura

In [63]:
model.save('botmedix_90PC.h5', hist)

Load pretrained model

In [64]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

with open('/content/drive/MyDrive/Colab Notebooks/dataset.json', 'r+') as f:
  data = json.load(f)

In [65]:
from tensorflow.keras.models import load_model

In [66]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
botmedix = load_model('botmedix.h5')

In [72]:
botmedix.evaluate(X, y)

615/615 [==============================] - 1s 1ms/step - loss: 0.1393 - accuracy: 0.9543


[0.13931460678577423, 0.9542682766914368]

In [67]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

In [68]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

In [69]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = botmedix.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x : x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

In [70]:
def get_response(intents_list, intents_json):
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
      result = random.choice(i['responses'])
      break
  return result

In [71]:
print("Go! Botmedix is working")

while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, data)
  print(res)

Go! Botmedix is working
itching, skin rash
1/1 [==============================] - 0s 86ms/step
It's probably fungal infection
muscle wasting, high fever
1/1 [==============================] - 0s 24ms/step
I woudl say the patient is sick of malaria
high fever, extra marital contacts
1/1 [==============================] - 0s 22ms/step
I think you are looking for malaria
patches in throat, high fever
1/1 [==============================] - 0s 23ms/step
I think you are looking for malaria
fatigue, restlessness
1/1 [==============================] - 0s 22ms/step
I think you are looking for hyperthyroidism
vomiting, sunken eyes
1/1 [==============================] - 0s 23ms/step
It's probably gastroenteritis


KeyboardInterrupt: ignored